# Домашнее задание 2 (5 баллов).

Все задания ниже имеют равный вес (5/16).

In [303]:
import pandas as pd
import numpy as np

import time
import functools
import zipfile
from pprint import pprint as pp
!wget https://github.com/hse-ds/iad-intro-ds/raw/master/2022/homeworks/Data.zip

--2022-02-09 20:00:48--  https://github.com/hse-ds/iad-intro-ds/raw/master/2022/homeworks/Data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2022/homeworks/Data.zip [following]
--2022-02-09 20:00:48--  https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2022/homeworks/Data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16727 (16K) [application/zip]
Saving to: ‘Data.zip.7’

Data.zip.7          100%[===================>]  16.33K  --.-KB/s    in 0s      

2022-02-09 20:00:48 (42.8 MB/s) - ‘Data.zip.7’ saved [16727/16727]



#### Описание данных

В папке Data находится информация о студентах. Всего 10 групп студентов. Файлы делятся на две категории:
    * Students_info_i - информация о студентах из группы i
    * Students_marks_i - оценки студентов из группы i за экзамены

### Одно из важных достоинств pandas $-$ это удобные методы реляционного взаимодействия с данными, аналогичные, например, возможностям SQL для слияния и конкатенации таблиц: merge, join, concat. Наличие готовых методов позволяет не реализовывать самостоятельно поэлементную обработку данных и оперировать сразу целыми таблицами данных.

Подробно об этих методах посмотрите тут: https://www.kaggle.com/residentmario/renaming-and-combining#Combining

#### Задание 1. Соберите всю информацию о студентах в одну таблицу df. В получившейся таблице должна быть информация и оценки всех студентов из всех групп. Напечатайте несколько строк таблицы для демонстрации результата.¶

In [304]:
students_info_dfs_lst = []
students_marks_dfs_lst = []

with zipfile.ZipFile("Data.zip") as z:
    print()
    for i in range(10):
        with z.open(f"Data/Students_info_{i}.csv") as f:
            students_info_dfs_lst.append(
                pd.read_csv(f, sep=",")
            )
        with z.open(f"Data/Students_marks_{i}.csv") as f:
            students_marks_dfs_lst.append(
            pd.read_csv(f, sep=",")
        )

students_info_df = pd.concat(students_info_dfs_lst, axis=0)
students_marks_df = pd.concat(students_marks_dfs_lst, axis=0)

students_info_and_marks_df = pd.merge(
    students_info_df, students_marks_df, on='index', how='left'
)

df = students_info_and_marks_df

df.replace(to_replace=["none"], value=np.nan, inplace=True)

df.head()

,index,gender,race/ethnicity,parental level of education,lunch,test preparation course,group,math score,reading score,writing score
0,0,female,group B,bachelor's degree,standard,NaN,group1,72,72,74
1,1,female,group C,some college,standard,completed,group1,69,90,88
2,2,female,group B,master's degree,standard,NaN,group1,90,95,93
3,3,male,group A,associate's degree,free/reduced,NaN,group1,47,57,44
4,4,male,group C,some college,standard,NaN,group1,76,78,75


#### Задание 2. Удалите столбец index у полученной таблицы. Напечатайте первые 10 строк таблицы.

In [305]:
df = df[df.columns.drop("index")]

df.head(10)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,group,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,NaN,group1,72,72,74
1,female,group C,some college,standard,completed,group1,69,90,88
2,female,group B,master's degree,standard,NaN,group1,90,95,93
3,male,group A,associate's degree,free/reduced,NaN,group1,47,57,44
4,male,group C,some college,standard,NaN,group1,76,78,75
5,female,group B,associate's degree,standard,NaN,group1,71,83,78
6,female,group B,some college,standard,completed,group1,88,95,92
7,male,group B,some college,free/reduced,NaN,group1,40,43,39
8,male,group D,high school,free/reduced,completed,group1,64,64,67
9,female,group B,high school,free/reduced,NaN,group1,38,60,50


#### Задание 3. Выведите на экран размеры полученной таблицы





In [306]:
df.shape

(1000, 9)

#### Задание 4. Выведите на экран статистические характеристики числовых столбцов таблицы (минимум, максимум, среднее значение, стандартное отклонение)

In [307]:
class ColumnProperties:
    """Класс для вычисление и вывода числовых характеристик столбцов таблицы.

    Характеристики:
    - Минимум,
    - Максимум,
    - Среднее значение,
    - Стандартное отклонение."""

    def __init__(self, column_name: str, table_df: pd.DataFrame) -> None:
        self.column_name = column_name
        self._column_df = table_df[column_name]
      
    def __str__(self) -> str:
        return f"ColumnProperties: {self.column_name}"
    
    def __repr__(self) -> str:
        return f"ColumnProperties('{self.column_name}')"

    def get_min_value(self) -> str:
        return self._column_df.min()
    
    def get_max_value(self) -> str:
        return self._column_df.max()

    def get_mean_value(self) -> str:
        return self._column_df.mean()
    
    def get_std_value(self) -> str:
        return self._column_df.std()
    
    def print_all_properties(self) -> None:
      print(f"""
                {self.column_name}:
      max: {self.get_max_value()}
      min: {self.get_min_value()}
      mean: {self.get_mean_value()}
      standart deviation: {self.get_std_value()}""")


def main():
    math_score = ColumnProperties(column_name='math score', table_df=df)
    reading_score = ColumnProperties(column_name='reading score', table_df=df)
    writing_score = ColumnProperties(column_name='writing score', table_df=df)

    math_score.print_all_properties()
    reading_score.print_all_properties()
    writing_score.print_all_properties()

#### способ 1

In [308]:
main()


                math score:
      max: 100
      min: 0
      mean: 66.089
      standart deviation: 15.163080096009468

                reading score:
      max: 100
      min: 17
      mean: 69.169
      standart deviation: 14.60019193725222

                writing score:
      max: 100
      min: 10
      mean: 68.054
      standart deviation: 15.195657010869642


#### способ 2

In [309]:
df[['math score', 'reading score', 'writing score']].max()

math score       100
reading score    100
writing score    100
dtype: int64

In [310]:
df[['math score', 'reading score', 'writing score']].min()

math score        0
reading score    17
writing score    10
dtype: int64

In [311]:
df[['math score', 'reading score', 'writing score']].mean()

math score       66.089
reading score    69.169
writing score    68.054
dtype: float64

In [312]:
df[['math score', 'reading score', 'writing score']].std()

math score       15.163080
reading score    14.600192
writing score    15.195657
dtype: float64

#### Задание 5. Проверьте, есть ли в таблице пропущенные значения

In [313]:
df_nulls = df.isnull().sum(axis=0)
df_nulls

gender                           0
race/ethnicity                   0
parental level of education      0
lunch                            0
test preparation course        642
group                            0
math score                       0
reading score                    0
writing score                    0
dtype: int64

In [314]:
df_nulls[['test preparation course']]

test preparation course    642
dtype: int64

**Да, есть**

#### Задание 6. Выведите на экран средние баллы студентов по каждому предмету (math, reading, writing)

In [315]:
df[['math score', 'reading score', 'writing score']].mean()

math score       66.089
reading score    69.169
writing score    68.054
dtype: float64

**Задание 7. Как зависят оценки от того, проходил ли студент курс для подготовки к сдаче экзамена (test preparation course)? Выведите на экран для каждого предмета в отдельности средний балл студентов, проходивших курс для подготовки к экзамену и не проходивших курс.**

In [316]:
df.loc[df['test preparation course'] == "completed"][['math score', 'reading score', 'writing score']].mean()

math score       69.695531
reading score    73.893855
writing score    74.418994
dtype: float64

In [317]:
df.loc[df['test preparation course'] != "completed"][['math score', 'reading score', 'writing score']].mean()

math score       64.077882
reading score    66.534268
writing score    64.504673
dtype: float64

Средняя оценка студентов не закончивших курс ниже

**Задание 8. Выведите на экран все различные значения из столбца lunch.**

In [318]:
df["lunch"].unique()

array(['standard', 'free/reduced'], dtype=object)

**Задание 9. Переименуйте колонку "parental level of education" в "education", а "test preparation course" в "test preparation" с помощью метода pandas rename**
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html

In [319]:
df = df.rename(columns={"parental level of education": "education", "test preparation course": "test preparation"})
df

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,NaN,group1,72,72,74
1,female,group C,some college,standard,completed,group1,69,90,88
2,female,group B,master's degree,standard,NaN,group1,90,95,93
3,male,group A,associate's degree,free/reduced,NaN,group1,47,57,44
4,male,group C,some college,standard,NaN,group1,76,78,75
...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,group10,88,99,95
996,male,group C,high school,free/reduced,NaN,group10,62,55,55
997,female,group C,high school,free/reduced,completed,group10,59,71,65
998,female,group D,some college,standard,completed,group10,68,78,77


**Зафиксируем минимальный балл для сдачи экзамена**

In [320]:
passmark = 50

**Задание 10. Ответьте на вопросы:**
    * Какая доля студентов сдала экзамен по математике (passmark >= 50)?
    * Какая доля студентов, проходивших курс подготовки к экзамену, сдала экзамен по математике?
    * Какая доля женщин, не проходивших курс подготовки к экзамену, не сдала экзамен по математике? 

In [321]:
df.loc[df['math score'] >= passmark].count()["gender"] / df.count()["gender"]

0.865

In [322]:
df.loc[(df['math score'] >= passmark) & (df['test preparation'] == "completed")].count()["gender"] / df.count()["gender"]

0.33

In [323]:
df.loc[(df['math score'] >= passmark) & (df['test preparation'] != "completed") & (df['gender'] == "female")].count()["gender"] / df.count()["gender"]

0.264

**Задание 11. С помощью groupby выполните задания ниже. Также выведите время выполнения каждого из заданий.**
    * Для каждой этнической группы выведите средний балл за экзамен по чтению
    * Для каждого уровня образования выведите минимальный балл за экзамен по письму

In [324]:
time_dict = {}

def view_function_timer():

    def decorator(func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            try:
                t0 = time.time()
                return func(*args, **kwargs)
            finally:
                t1 = time.time()

                time_dict[func.__name__] = t1 - t0
                
                print(
                    f"""
                    Function: {func.__name__}
                    Took: {1000 * (t1 - t0)} ms"""
                )
        return inner

    return decorator

In [325]:
@view_function_timer()
def task_11_1():
   return df.groupby(by="race/ethnicity").mean()["reading score"]


@view_function_timer()
def task_11_2():
   return df.groupby(by="education").mean()["writing score"]

print(f"""
--------------------------------------------------------------------------------

    task 1 result: 
    
    {task_11_1()}


--------------------------------------------------------------------------------

    task 2 result: 
    
    {task_11_2()}


--------------------------------------------------------------------------------
""")


                    Function: task_11_1
                    Took: 4.0416717529296875 ms

                    Function: task_11_2
                    Took: 2.0041465759277344 ms

--------------------------------------------------------------------------------

    task 1 result: 
    
    race/ethnicity
group A    64.674157
group B    67.352632
group C    69.103448
group D    70.030534
group E    73.028571
Name: reading score, dtype: float64


--------------------------------------------------------------------------------

    task 2 result: 
    
    education
associate's degree    69.896396
bachelor's degree     73.381356
high school           62.448980
master's degree       75.677966
some college          68.840708
some high school      64.888268
Name: writing score, dtype: float64


--------------------------------------------------------------------------------



**Задание 12. Выполните задание 11 с помощью циклов. Сравните время выполнения.**

In [326]:
@view_function_timer()
def task_12_1():
  groups_mean = []
  for group in df["education"].unique():
     groups_mean.append(
         df.loc[df['race/ethnicity'] == group].mean()[["reading score"]])
  return groups_mean


@view_function_timer()
def task_12_2():
  education_mean = []
  for education in df["education"].unique():
     education_mean.append(
         df.loc[df['education'] == education].mean()[["writing score"]])
  return education_mean


print(f"""
--------------------------------------------------------------------------------

    task 1 result: 
    
    {task_12_1()}


--------------------------------------------------------------------------------

    task 2 result: 
    
    {task_12_2()}


--------------------------------------------------------------------------------
""")


                    Function: task_12_1
                    Took: 11.164665222167969 ms

                    Function: task_12_2
                    Took: 17.500638961791992 ms

--------------------------------------------------------------------------------

    task 1 result: 
    
    [reading score    NaN
dtype: object, reading score    NaN
dtype: object, reading score    NaN
dtype: object, reading score    NaN
dtype: object, reading score    NaN
dtype: object, reading score    NaN
dtype: object]


--------------------------------------------------------------------------------

    task 2 result: 
    
    [writing score    73.381356
dtype: float64, writing score    68.840708
dtype: float64, writing score    75.677966
dtype: float64, writing score    69.896396
dtype: float64, writing score    62.44898
dtype: float64, writing score    64.888268
dtype: float64]


--------------------------------------------------------------------------------



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  from ipykernel import kernelapp as app


In [327]:
time_task_11_1 = time_dict['task_11_1']
time_task_12_1 = time_dict['task_12_1']

time_task_11_2 = time_dict['task_11_2']
time_task_12_2 = time_dict['task_12_2']

print(f"""
Task 1
Первый способ быстрее в {time_task_12_1 / time_task_11_1}.

Task 2
Первый способ быстрее в {time_task_12_2 / time_task_11_2}.
""")


Task 1
Первый способ быстрее в 2.7623879188296367.

Task 2
Первый способ быстрее в 8.732215084463478.



**Задание 13. Выведите на экран средние баллы студентов по каждому предмету в зависимости от пола и уровня образования. То есть должно получиться количество групп, равных 2 * (число уровней образования), и для каждой такой группы выыведите средний балл по каждому из предметов.**

Это можно сделать с помощью сводных таблиц (pivot_table):

https://www.kaggle.com/kamilpolak/tutorial-how-to-use-pivot-table-in-pandas

In [328]:
grouped = pd.pivot_table(df,index=['gender','education'])
grouped

math score  reading score  writing score
gender education                                                   
female associate's degree   65.250000      74.120690      74.000000
       bachelor's degree    68.349206      77.285714      78.380952
       high school          59.351064      68.202128      66.691489
       master's degree      66.500000      76.805556      77.638889
       some college         65.406780      73.550847      74.050847
       some high school     59.296703      69.109890      68.285714
male   associate's degree   70.764151      67.433962      65.405660
       bachelor's degree    70.581818      68.090909      67.654545
       high school          64.705882      61.480392      58.539216
       master's degree      74.826087      73.130435      72.608696
       some college         69.009259      64.990741      63.148148
       some high school     67.840909      64.693182      61.375000

#### Задание 14. Сколько студентов успешно сдали экзамен по математике?

Создайте новый столбец в таблице df под названием Math_PassStatus и запишите в него F, если студент не сдал экзамен по математике (балл за экзамен < passmark), и P иначе.

Посчитайте количество студентов, сдавших и не сдавших экзамен по математике.

Сделайте аналогичные шаги для экзаменов по чтению и письму.

In [329]:
df["Math_PassStatus"] = df['math score'] >= passmark
df["Math_PassStatus"] = df["Math_PassStatus"].replace({True: "P", False: "F"})
print("Сдали экзамен по математике:", df.loc[df['Math_PassStatus'] == "P"]["gender"].count())

df["Writing_PassStatus"] = df['math score'] >= passmark
df["Writing_PassStatus"] = df["Writing_PassStatus"].replace({True: "P", False: "F"})
print("Сдали экзамен по письму:", df.loc[df['Writing_PassStatus'] == "P"]["gender"].count())

df["Reading_PassStatus"] = df['math score'] >= passmark
df["Reading_PassStatus"] = df["Reading_PassStatus"].replace({True: "P", False: "F"})
print("Сдали экзамен по чтению:", df.loc[df['Reading_PassStatus'] == "P"]["gender"].count())


Сдали экзамен по математике: 865
Сдали экзамен по письму: 865
Сдали экзамен по чтению: 865


#### Задание 15. Сколько студентов успешно сдали все экзамены?

Создайте столбец OverAll_PassStatus и запишите в него для каждого студента 'F', если студент не сдал хотя бы один из трех экзаменов, а иначе 'P'.

Посчитайте количество студентов, которые сдали все экзамены.

In [330]:
df["OverAll_PassStatus"] = (df['Math_PassStatus'] == "F") | (df['Writing_PassStatus'] == "F") | (df['Reading_PassStatus'] == "F")
df["OverAll_PassStatus"] = df["OverAll_PassStatus"].replace({True: "F", False: "P"})

print("Сдали все экзамены:", df.loc[df['Reading_PassStatus'] == "P"]["gender"].count())

Сдали все экзамены: 865


#### Задание 16. Переведем баллы в оценки

### Система перевода баллов в оценки
####    больше 90 = A
####      80-90 = B
####      70-80 = C
####      60-70 = D
####      50-60 = E
####    меньше 50 = F (Fail)

Создайте вспомогательную функцию, которая будет по среднему баллу за три экзамена выставлять оценку студенту по данным выше критериям.

Создайте столбец Grade и запишите в него оценку каждого студента.

Выведите количество студентов, получивших каждую из оценок.

**В случае, если средний балл попадает на границу между оценками (т.е. равен ровно 60, 70 или 80 баллов), вы можете интерпретировать условие на своё усмотрение (т.е. можете поставить за 60 баллов оценку D, а можете - E).**

In [335]:
def get_grade(average_mark: float) -> str:
    if (average_mark < 50):
        grade = "F (Fail)"
    elif average_mark < 60:
        grade = "E"
    elif average_mark < 70:
        grade = "D"
    elif average_mark < 80:
        grade = "C"
    elif average_mark < 90:
        grade = "B"
    else:
        grade = "A"
    return grade

math_score = df['math score']
reading_score = df['reading score']
writing_score = df['writing score']

average_score = (math_score + reading_score + writing_score) / 3

grade = average_score.apply(lambda x: get_grade(x))

df["Grade"] = grade

df.loc[(df['Math_PassStatus'] != df['Writing_PassStatus']) & (df['Math_PassStatus'] != df['Reading_PassStatus'])]["gender"].count()

0

In [336]:
df

,gender,race/ethnicity,education,lunch,test preparation,group,math score,reading score,writing score,Math_PassStatus,Writing_PassStatus,Reading_PassStatus,OverAll_PassStatus,Grade
0,female,group B,bachelor's degree,standard,NaN,group1,72,72,74,P,P,P,P,C
1,female,group C,some college,standard,completed,group1,69,90,88,P,P,P,P,B
2,female,group B,master's degree,standard,NaN,group1,90,95,93,P,P,P,P,A
3,male,group A,associate's degree,free/reduced,NaN,group1,47,57,44,F,F,F,F,F (Fail)
4,male,group C,some college,standard,NaN,group1,76,78,75,P,P,P,P,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,group10,88,99,95,P,P,P,P,A
996,male,group C,high school,free/reduced,NaN,group10,62,55,55,P,P,P,P,E
997,female,group C,high school,free/reduced,completed,group10,59,71,65,P,P,P,P,D
998,female,group D,some college,standard,completed,group10,68,78,77,P,P,P,P,C
